In [1]:
# Libraries for data preparation & visualization
import numpy as np
import pandas as pd

# Ignore printing warnings for general readability
import warnings 
warnings.filterwarnings("ignore")

# pip install scikit-surprise
# Importing libraries for model building & evaluation
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import accuracy
import random

In [2]:
book_path = "./dataset/BX-Books"

In [3]:
# Fix the &amp; and other HTML escape sequences
import html
import os
print(os.getcwd())

raw_book_path = book_path
book_path = "./dataset/BX-Books-HTMLfixed"
if not os.path.isfile(f'{book_path}.csv'): # If doesn't exist
    with open(f'{raw_book_path}.csv', 'r', encoding='latin-1') as f, open(f'{book_path}.csv', 'w') as g:
        content = html.unescape(f.read())
        g.write(content)
        print("Fixed file {0}.csv".format(raw_book_path))

C:\Users\Legion\Downloads\b\1INTRO2AI\GithubDesktop\book-rec


In [4]:
# Loading the dataset 
def loaddata(filename):
    df = pd.read_csv(f'{filename}.csv',sep=';', encoding='latin-1', escapechar='\\',\
                    error_bad_lines=False, warn_bad_lines=False,)
    return df

# Use your path in local
book   = loaddata(book_path)
user   = loaddata("./dataset/BX-Users")
rating = loaddata("./dataset/BX-Book-Ratings")

In [5]:
rating_users = rating['User-ID'].value_counts().reset_index().\
               rename({'Index':'User-ID','User-ID':'Rating'}, axis=1)
rating_books = rating['ISBN'].value_counts().reset_index().\
               rename({'Index':'ISBN','ISBN':'Rating'}, axis=1)
# In order to avoid rating bias & for making good recommendations, limit the dataset to only those
# users that have made at least 100 ratings & books that have received at least 50 ratings

rating = rating[rating['User-ID'].isin(rating_users[rating_users['Rating']>=100]['index'])]
rating = rating[rating['ISBN'].isin(rating_books[rating_books['Rating']>=50]['index'])]

rating

,User-ID,ISBN,Book-Rating
413,276925,002542730X,10
426,276925,0316666343,0
427,276925,0345391810,0
429,276925,0385504209,8
448,276925,0679745580,0
...,...,...,...
1149604,276680,0743486226,6
1149616,276680,0812969812,0
1149628,276680,1573222267,0
1149629,276680,1573229083,7


In [6]:
# For the recommendation system, it is prefered to have the book titles rather than ISBN for easier interpretation

rating = rating.merge(book, on="ISBN")[['User-ID','ISBN','Book-Rating','Book-Title']] # merging with the book dataframe
rating               

,User-ID,ISBN,Book-Rating,Book-Title
0,276925,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...
1,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...
2,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...
3,10030,002542730X,7,Politically Correct Bedtime Stories: Modern Ta...
4,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...
...,...,...,...,...
118771,238781,0743406184,10,If Only It Were True
118772,246156,0743406184,0,If Only It Were True
118773,246617,0743406184,0,If Only It Were True
118774,274308,0743406184,0,If Only It Were True


In [7]:
ratings_explicit=rating[rating['Book-Rating']!=0]
ratings_implicit=rating[rating['Book-Rating']==0]
print(ratings_explicit.shape)
print(ratings_implicit.shape)

(32125, 4)
(86651, 4)


In [8]:
# creating a surprise object

reader = Reader(rating_scale=(1, 10))
data_nonzero   = Dataset.load_from_df(ratings_explicit[['User-ID','ISBN','Book-Rating']], reader)
data  = Dataset.load_from_df(rating[['User-ID','ISBN','Book-Rating']], reader)

# Split the data into training & testing sets. 

raw_ratings_nonzero = data_nonzero.raw_ratings
random.shuffle(raw_ratings_nonzero)                 # shuffle dataset

threshold   = int(len(raw_ratings_nonzero)*0.8)

train_raw_ratings = raw_ratings_nonzero[:threshold] # 80% of data is trainset
test_raw_ratings  = raw_ratings_nonzero[threshold:] # 20% of data is testset

data_nonzero.raw_ratings = train_raw_ratings        # data is now the trainset
trainset         = data_nonzero.build_full_trainset() 
testset          = data_nonzero.construct_testset(test_raw_ratings)


In [9]:
# Trying KNN (K-Nearest Neighbors) with data

models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()] 
results = {}

for model in models:
    # perform 5 fold cross validation
    # evaluation metrics: mean absolute error & root mean square error
    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)  
    
    # storing the average score across the 5 fold cross validation for each model
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

In [10]:
performance_df = pd.DataFrame.from_dict(results)
print("Model Performance: \n")
performance_df.T.sort_values(by='RMSE')

Model Performance: 



,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,2.708886,3.383551,0.915324,1.690223
knns.KNNWithMeans,2.701036,3.386168,0.828715,1.473749
knns.KNNWithZScore,2.689598,3.411793,0.888407,1.790256
knns.KNNBasic,2.826441,3.576992,0.994729,1.396530


In [11]:
# Trying KNN (K-Nearest Neighbors) with nonzero rating data

models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()] 
results = {}

for model in models:
    # perform 5 fold cross validation
    # evaluation metrics: mean absolute error & root mean square error
    CV_scores = cross_validate(model, data_nonzero, measures=["MAE","RMSE"], cv=5, n_jobs=-1)  
    
    # storing the average score across the 5 fold cross validation for each model
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

In [12]:
performance_df = pd.DataFrame.from_dict(results)
print("Model Performance: \n")
performance_df.T.sort_values(by='RMSE')

Model Performance: 



,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,1.297845,1.721187,0.114826,0.139030
knns.KNNWithMeans,1.282748,1.727952,0.101822,0.092029
knns.KNNWithZScore,1.276225,1.732670,0.147434,0.094620
knns.KNNBasic,1.433585,1.879371,0.092421,0.101632


In [13]:
# Hyperparameter tuning - KNNBasic with data_nonzero

param_grid = { 'sim_options' : {'name': ['msd','cosine','pearson','pearson_baseline'], \
                                'min_support': [1,5], \
                                'user_based': [False, True]}
             }

gridsearchKNNBasic = GridSearchCV(KNNBasic, param_grid, measures=['mae', 'rmse'], cv=5, n_jobs=-1)
                                    
gridsearchKNNBasic.fit(data_nonzero)

print(f'MAE Best Parameters:  {gridsearchKNNBasic.best_params["mae"]}')
print(f'MAE Best Score:       {gridsearchKNNBasic.best_score["mae"]}\n')

print(f'RMSE Best Parameters: {gridsearchKNNBasic.best_params["rmse"]}')
print(f'RMSE Best Score:      {gridsearchKNNBasic.best_score["rmse"]}\n')

MAE Best Parameters:  {'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}
MAE Best Score:       1.2408618952270847

RMSE Best Parameters: {'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}
RMSE Best Score:      1.6878894121650745



In [14]:
# Hyperparameter tuning - KNNWithMeans with data_nonzero

param_grid = { 'sim_options' : {'name': ['msd','cosine','pearson','pearson_baseline'], \
                                'min_support': [1,5], \
                                'user_based': [False, True]}
             }

gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['mae', 'rmse'], cv=5, n_jobs=-1)
                                    
gridsearchKNNWithMeans.fit(data_nonzero)

print(f'MAE Best Parameters:  {gridsearchKNNWithMeans.best_params["mae"]}')
print(f'MAE Best Score:       {gridsearchKNNWithMeans.best_score["mae"]}\n')

print(f'RMSE Best Parameters: {gridsearchKNNWithMeans.best_params["rmse"]}')
print(f'RMSE Best Score:      {gridsearchKNNWithMeans.best_score["rmse"]}\n')

MAE Best Parameters:  {'sim_options': {'name': 'pearson_baseline', 'min_support': 5, 'user_based': True}}
MAE Best Score:       1.2601334826742108

RMSE Best Parameters: {'sim_options': {'name': 'pearson_baseline', 'min_support': 5, 'user_based': True}}
RMSE Best Score:      1.686990863080951



In [15]:
# Hyperparameter tuning - KNNWithZScore

param_grid = { 'sim_options' : {'name': ['msd','cosine','pearson','pearson_baseline'], \
                                'min_support': [1,5], \
                                'user_based': [False, True]}
             }

gridsearchKNNZScore = GridSearchCV(KNNWithZScore, param_grid, measures=['mae', 'rmse'], \
                                      cv=5, n_jobs=-1)
                                    
gridsearchKNNZScore.fit(data_nonzero)

print(f'MAE Best Parameters:  {gridsearchKNNZScore.best_params["mae"]}')
print(f'MAE Best Score:       {gridsearchKNNZScore.best_score["mae"]}\n')

print(f'RMSE Best Parameters: {gridsearchKNNZScore.best_params["rmse"]}')
print(f'RMSE Best Score:      {gridsearchKNNZScore.best_score["rmse"]}\n')

MAE Best Parameters:  {'sim_options': {'name': 'pearson_baseline', 'min_support': 5, 'user_based': True}}
MAE Best Score:       1.2489907277236454

RMSE Best Parameters: {'sim_options': {'name': 'pearson_baseline', 'min_support': 5, 'user_based': True}}
RMSE Best Score:      1.6734797865691717



In [16]:
# Hyperparameter tuning - KNNBaseLine

param_grid = { 'sim_options' : {'name': ['msd','cosine','pearson','pearson_baseline'], \
                                'min_support': [1,5], \
                                'user_based': [False, True]}
             }

gridsearchKNNBaseLine = GridSearchCV(KNNBaseline, param_grid, measures=['mae', 'rmse'], \
                                      cv=5, n_jobs=-1)
                                    
gridsearchKNNBaseLine.fit(data_nonzero)

print(f'MAE Best Parameters:  {gridsearchKNNBaseLine.best_params["mae"]}')
print(f'MAE Best Score:       {gridsearchKNNBaseLine.best_score["mae"]}\n')

print(f'RMSE Best Parameters: {gridsearchKNNBaseLine.best_params["rmse"]}')
print(f'RMSE Best Score:      {gridsearchKNNBaseLine.best_score["rmse"]}\n')

MAE Best Parameters:  {'sim_options': {'name': 'pearson_baseline', 'min_support': 5, 'user_based': False}}
MAE Best Score:       1.1862312008420144

RMSE Best Parameters: {'sim_options': {'name': 'pearson_baseline', 'min_support': 5, 'user_based': False}}
RMSE Best Score:      1.5550963192741871



In [17]:
best_options       = gridsearchKNNWithMeans.best_params["rmse"]["sim_options"]
print(best_options)

# # We'll use the KNNWithMeans
algo = KNNWithMeans(sim_options=best_options)

# Run 5-fold cross-validation and print results
CV_score = cross_validate(algo, data_nonzero, measures=["RMSE", "MAE"], cv=5, verbose=True)

{'name': 'pearson_baseline', 'min_support': 5, 'user_based': True}
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6759  1.6582  1.7239  1.7125  1.6944  1.6930  0.0238  
MAE (testset)     1.2608  1.2421  1.2796  1.2801  1.2659  1.2657  0.0140  
Fit time          0.15    0.19    0.14    0.14    0.14    0.

In [18]:
best_options       = gridsearchKNNBaseLine.best_params["rmse"]["sim_options"]
print(best_options)

# # We'll use the KNNBaseline
algo = KNNBaseline(sim_options=best_options)

# Run 5-fold cross-validation and print results
CV_score = cross_validate(algo, data_nonzero, measures=["RMSE", "MAE"], cv=5, verbose=True)

{'name': 'pearson_baseline', 'min_support': 5, 'user_based': False}
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5301  1.5696  1.5464  1.5930  1.5460  1.5570  0.0220  
MAE (testset)     1.1612  1.1926  1.1940  1.2036  1.1862  1.1875  0.0143  
Fit time          0.22    0.23    0.24    0.26    0.24    0.

In [19]:
best_options       = gridsearchKNNBasic.best_params["rmse"]["sim_options"]
print(best_options)

# # We'll use the KNNBasic
algo = KNNBasic(sim_options=best_options)

# Run 5-fold cross-validation and print results
CV_score = cross_validate(algo, data_nonzero, measures=["RMSE", "MAE"], cv=5, verbose=True)

{'name': 'cosine', 'min_support': 1, 'user_based': False}
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6900  1.6677  1.7569  1.6738  1.6747  1.6926  0.0330  
MAE (testset)     1.2518  1.2268  1.2940  1.2323  1.2348  1.2479  0.0245  
Fit time          0.21    0.24    0.24    0.24    0.19    0.22    0.02    
Test time         0.15    0.22    0.16    0.18    0.15    0.17    0.03    


In [20]:
# Model fit & prediction - KNNWithMeans

best_withmeans_sim_options = gridsearchKNNWithMeans.best_params["rmse"]["sim_options"]
final_model = KNNWithMeans(sim_options=best_withmeans_sim_options)

# Fitting the model on trainset & predicting on testset, printing test accuracy
pred = final_model.fit(trainset).test(testset)

print(f'\nUnbiased Testing Performance:')
print(f'MAE: {accuracy.mae(pred)}, RMSE: {accuracy.rmse(pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Unbiased Testing Performance:
MAE:  1.2695
RMSE: 1.7266
MAE: 1.2695026204297546, RMSE: 1.7266287096350093


In [21]:
# Model fit & prediction - KNNBaseline
# Best options for this model
best_baseline_sim_options = gridsearchKNNBaseLine.best_params["rmse"]["sim_options"]
final_model = KNNBaseline(sim_options=best_baseline_sim_options)

# Fitting the model on trainset & predicting on testset, printing test accuracy
pred = final_model.fit(trainset).test(testset)

print(f'\nUnbiased Testing Performance:')
print(f'MAE: {accuracy.mae(pred)}, RMSE: {accuracy.rmse(pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Unbiased Testing Performance:
MAE:  1.1667
RMSE: 1.5567
MAE: 1.1666666780747936, RMSE: 1.556670254557254


In [22]:
# Model fit & prediction - KNNBasic

best_basic_sim_options = gridsearchKNNBasic.best_params["rmse"]["sim_options"]
final_model = KNNBasic(sim_options=best_basic_sim_options)

# Fitting the model on trainset & predicting on testset, printing test accuracy
pred = final_model.fit(trainset).test(testset)

print(f'\nUnbiased Testing Performance:')
print(f'MAE: {accuracy.mae(pred)}, RMSE: {accuracy.rmse(pred)}')

Computing the cosine similarity matrix...
Done computing similarity matrix.

Unbiased Testing Performance:
MAE:  1.2170
RMSE: 1.6625
MAE: 1.21697073549722, RMSE: 1.6624608376920034


<p>We can see that KNNBaseline performs best.</p>

In [23]:
set_userID = set([i[0] for i in train_raw_ratings])
def generate_recommendationsKNN(similarity_matrix, userID=13552, like_recommend=40, get_recommend =10):
    if (userID not in set_userID) or type(userID) is not int:
        print("User id should be a valid integer from this list : \n\n {}".format(sorted(set_userID)))
        return []
    
    ''' This function generates "get_recommend" number of book recommendations using 
        KNNWithMeans & item based filtering. The function needs as input three 
        different parameters:
        (1) userID i.e., userID for which recommendations need to be generated 
        (2) like_recommend i.e., number of top recommendations for the userID to be 
        considered for making recommendations 
        (3) get_recommend i.e., number of recommendations to generate for the userID
        Default values are: userID=13552, like_recommend=5, get_recommend=10
    '''
    
    userID      = trainset.to_inner_uid(userID)    # converts the raw userID to innerID
    userRatings = trainset.ur[userID]              # method .ur takes user innerID & 
                                                   # returns back user ratings
    
    
    # userRatings is a list of tuples [(,),(,),(,)..]. Each tuple contains item & rating
    # given by the user for that item. Next, the tuples will be sorted within the list 
    # in decreasing order of rating. Then top 'like_recommend' items & ratings are extracted
    
    temp_df = pd.DataFrame(userRatings).sort_values(by=1, ascending=False).\
              head(like_recommend)
    userRatings = temp_df.to_records(index=False) 
    
    # for each (item,rating) in top like_recommend user items, multiply the user rating for
    # the item with the similarity score (later is obtained from item similarity_matrix) for
    # all items. This helps calculate the weighted rating for all items. The weighted ratings 
    # are added & divided by sum of weights to estimate rating the user would give an item
    
    recommendations   = {}

    for user_top_item, user_top_item_rating  in userRatings:

        all_item_indices          =   list(pd.DataFrame(similarity_matrix)[user_top_item].index)
        all_item_weighted_rating  =   list(pd.DataFrame(similarity_matrix)[user_top_item].values*\
                                          user_top_item_rating)
        
        all_item_weights          =   list(pd.DataFrame(similarity_matrix)[user_top_item].values)
        
        
        # All items & final estimated ratings are added to a dictionary called recommendations
        
        for index in range(len(all_item_indices)):
            if index in recommendations:
                # sum of weighted ratings
                recommendations[index] += all_item_weighted_rating[index]        
            else:                        
                recommendations[index]  = all_item_weighted_rating[index]

    
    for index in range(len(all_item_indices)):                               
            if all_item_weights[index]  !=0:
                # final ratings (sum of weighted ratings/sum of weights)
                recommendations[index]   =recommendations[index]/\
                                          (all_item_weights[index]*like_recommend)
                      

    # convert dictionary recommendations to a be a list of tuples [(,),(,),(,)]
    # with each tuple being an item & estimated rating user would give that item
    # sort the tuples within the list to be in decreasing order of estimated ratings

    temp_df = pd.Series(recommendations).reset_index().sort_values(by=0, ascending=False)
    recommendations = list(temp_df.to_records(index=False))
    
    # return get_recommend number of recommedations (only return items the user 
    # has not previously rated)
    
    final_recommendations = []
    count = 0
    
    for item, score in recommendations:
        flag = True
        for userItem, userRating in trainset.ur[userID]:
            if item == userItem: 
                flag = False       # If item in recommendations has not been rated by user, 
                break              # add to final_recommendations
        if flag == True:
            final_recommendations.append(trainset.to_raw_iid(item)) 
            count +=1              # trainset has the items stored as inner id,  
                                   # convert to raw id & append 
            
        if count > get_recommend:  # Only get 'get_recommend' number of recommendations
            break
    
    return(final_recommendations)

In [24]:
def printRecommendations(similarity_matrix, user_id, k, get_top):
    recommendationsKNN = generate_recommendationsKNN(similarity_matrix, userID=user_id, like_recommend=k, get_recommend=get_top)

    print("\nRecommended Books for user {0} (item-based):".format(user_id))
    red = pd.DataFrame(recommendationsKNN,columns = ['ISBN'])
    red_ = red.merge(book, on="ISBN")[['ISBN','Book-Title']]
    return red_

In [25]:
# Set the user and other values
base_user_id = 13552
base_k = 40
base_recommend = 10

In [26]:
# Compute item based similarity matrix
# KNNBasic
best_basic_sim_options['user_based'] = False
similarities = KNNBasic(sim_options = best_basic_sim_options).fit(trainset).\
                    compute_similarities() 

printRecommendations(similarities, base_user_id, base_k, base_recommend)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.

Recommended Books for user 13552 (item-based):


,ISBN,Book-Title
0,0440213525,The Client
1,014028009X,Bridget Jones's Diary
2,055321313X,Anne of Green Gables (Anne of Green Gables Nov...
3,0671004530,On the Street Where You Live
4,0553572997,The Alienist
5,044021145X,The Firm
6,0440234743,The Testament
7,0671004573,Before I Say Good-Bye
8,0671525743,Night Whispers
9,0060502258,The Divine Secrets of the Ya-Ya Sisterhood: A ...


In [27]:
# Compute item based similarity matrix
# KNNWithMeans
best_withmeans_sim_options['user_based'] = False
similarities = KNNWithMeans(sim_options = best_withmeans_sim_options).fit(trainset).\
                    compute_similarities()

printRecommendations(similarities, base_user_id, base_k, base_recommend)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Recommended Books for user 13552 (item-based):


,ISBN,Book-Title
0,0440213525,The Client
1,044021145X,The Firm
2,0440226430,Summer Sisters
3,0312195516,The Red Tent (Bestselling Backlist)
4,0440221471,The Runaway Jury
5,0440241073,The Summons
6,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel
7,0786868716,The Five People You Meet in Heaven
8,0515136379,Key of Knowledge (Key Trilogy (Paperback))
9,080410526X,All I Really Need to Know


In [29]:
# Compute item based similarity matrix
# KNNBaseline
best_baseline_sim_options['user_based'] = False
similarities = KNNBaseline(sim_options = best_baseline_sim_options).fit(trainset).\
                    compute_similarities() 

printRecommendations(similarities, base_user_id, base_k, base_recommend)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.

Recommended Books for user 13552 (item-based):


,ISBN,Book-Title
0,0440213525,The Client
1,044021145X,The Firm
2,0440226430,Summer Sisters
3,0312195516,The Red Tent (Bestselling Backlist)
4,0440221471,The Runaway Jury
5,0440241073,The Summons
6,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel
7,0786868716,The Five People You Meet in Heaven
8,0515136379,Key of Knowledge (Key Trilogy (Paperback))
9,080410526X,All I Really Need to Know


# Using sklearn NearestNeighbor

In [34]:
used_rating = ratings_explicit

ratings_matrix = used_rating.pivot(index = 'User-ID', columns = 'ISBN', values = 'Book-Rating')
print(ratings_matrix.shape)
ratings_matrix = ratings_matrix.fillna(0)

ratings_matrix

(1684, 2160)


ISBN,000649840X,0007110928,002026478X,0020442203,002542730X,0028604199,006000438X,0060008032,0060008776,006001203X,...,1860492592,1878424319,1885171080,1931561648,3257228007,3257229534,3404148665,3423202327,3442541751,3492045170
User-ID,,,,,,,,,,,,,,,,,,,,,
254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
277639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
278137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# Importing libraries for model building & evaluation 
from sklearn.neighbors import NearestNeighbors
import sklearn
import re

In [36]:
# KNN Function
#setting default variables
default_k = base_k
default_metric='cosine'
default_algorithm = 'brute'
print(default_k)

40


In [37]:
def findksimilarusers(user_id, rate_matrix, metric=default_metric, algo=default_algorithm, k=default_k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = algo)
    model_knn.fit(rate_matrix.values)
    
    loc = rate_matrix.index.get_loc(user_id)
    # Return k nearest neighbors' indices and distances to the user_id
    distances, indices = model_knn.kneighbors(rate_matrix.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    
    # Get similarity level by (1 - differences)
    similarities = 1 - distances.flatten()
    indices = indices.flatten()
    
    return similarities, indices

In [38]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, rate_matrix, similarities, indices):
    prediction = 0
    similarity_sum = np.sum(similarities) - 1 # Minus the current user (similarity = 1)
    weighted_sum = 0
    product = 1

    item_loc = rate_matrix.columns.get_loc(item_id)
    user_loc = rate_matrix.index.get_loc(user_id)
    mean_rating = rate_matrix.iloc[user_loc, :].mean()

    # For each similar user:
    for i in range(0, len(indices)):
        if indices[i] == user_loc:
            continue;
        else:
            # Get the rating differences from the mean (of other items), multiply by similarities with the selected user
            ratings_diff = rate_matrix.iloc[indices[i], item_loc] - np.mean(rate_matrix.iloc[indices[i],:])
            product = ratings_diff * (similarities[i])
            weighted_sum = weighted_sum + product
    
    # weighted_sum = summation< (x - mean) * sim > / sumsim
                                                     
    #in case of very sparse datasets, using correlation metric for collaborative based approach may give negative ratings
    #which are handled here as below
    if prediction <= 0:
        prediction = 1   
    elif prediction >10:
        prediction = 10
    
    # Calculate rating by weighted mean of ratings and similarity score of k similar users
    prediction = int(round(mean_rating + (weighted_sum/similarity_sum)))
    # print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [39]:
def recommendedItem(user_id, rate_matrix, metric=default_metric, algorithm=default_algorithm, k=default_k):
    if (user_id not in rate_matrix.index.values) or type(user_id) is not int:
        print("User id should be a valid integer from this list : \n\n {}".format(re.sub('[\[\]]', '', np.array_str(rate_matrix.index.values))))
    else:
        prediction = []
        # Similar users based on cosine similarity
        similarities, indices = findksimilarusers(user_id, rate_matrix, metric, algorithm, k)
        
        total_book = rate_matrix.shape[1]
        for i in range(total_book):
            item_id = str(rate_matrix.columns[i])
            if (rate_matrix[item_id][user_id] != 0): # Not rated already
                print("\r{0:<0.2f}% Finished".format(i*100/total_book), end='')
                prediction.append(predict_userbased(user_id, item_id, rate_matrix, similarities, indices))
            else:                    
                prediction.append(-1) # For books that user already rated

        print("\r100.00% Finished")
        print("...")
        
        prediction = pd.Series(prediction)
        prediction = prediction.sort_values(ascending = False)
        
        # rmse_score = rmse_on_test_set(rate_matrix, similarities, indices)
        # print("RMSE Score: ", rmse_score)
        
        recommended = prediction[:10]
        print("\nk = {0}".format(k))
        print("For the User-based ({0} - {1}) approach, the following books are recommended to user {2}: \n".\
              format(algorithm, metric, user_id))
        for i in range(len(recommended)):
            print("{0}. Rated {1}: {2}".format(i+1, prediction.values[i], book['Book-Title'][recommended.index[i]]))

In [40]:
metric = 'cosine'
algorithm = 'brute'
recommendedItem(base_user_id, ratings_matrix, metric, algorithm, base_k)

100.00% Finished
...

k = 40
For the User-based (brute - cosine) approach, the following books are recommended to user 13552: 

1. Rated 3: The Power of Myth (Illustrated Edition)
2. Rated 3: Tangle Box
3. Rated 2: All That Glitters (Avalon, 2)
4. Rated 2: The Brethren
5. Rated 2: RUSSIA HOUSE, THE
6. Rated 2: The Internet Church
7. Rated 2: The Wooden Nickel: A Novel
8. Rated 2: The Kalahari Typing School for Men : More from the No. 1 Ladies' Detective Agency
9. Rated 2: Only The River Runs Free : A Novel (Galway Chronicles (Paperback))
10. Rated 2: Patriot Games


In [41]:
metric = 'euclidean'
algorithm = 'brute'
recommendedItem(base_user_id, ratings_matrix, metric, algorithm, base_k)

100.00% Finished
...

k = 40
For the User-based (brute - euclidean) approach, the following books are recommended to user 13552: 

1. Rated 2: Echoes
2. Rated 1: My Teacher is an Alien
3. Rated 1: Every Breath You Take : A True Story of Obsession, Revenge, and Murder
4. Rated 1: Atonement
5. Rated 1: Rachel's Tears: The Spiritual Journey of Columbine Martyr Rachel Scott
6. Rated 1: The Golden Compass (His Dark Materials, Book 1)
7. Rated 1: Welcome to Higby : A Novel
8. Rated 1: Metropolis (Bfi Film Classics, 54)
9. Rated 1: The Internet Church
10. Rated 1: Espurnes de sang (La Negra)


In [42]:
metric = 'correlation'
algorithm = 'brute'
recommendedItem(base_user_id, ratings_matrix, metric, algorithm, base_k)

100.00% Finished
...

k = 40
For the User-based (brute - correlation) approach, the following books are recommended to user 13552: 

1. Rated 3: The Power of Myth (Illustrated Edition)
2. Rated 2: All That Glitters (Avalon, 2)
3. Rated 2: Trial by Fire
4. Rated 2: The Diary of Ellen Rimbauer: My Life at Rose Red
5. Rated 2: My Teacher is an Alien
6. Rated 2: Tangle Box
7. Rated 2: Atonement
8. Rated 2: French Lieutenant's Woman
9. Rated 2: Driving Mr. Albert: A Trip Across America With Einstein's Brain
10. Rated 2: The Brethren
